In [2]:
# Notebook ssetup
import os
import numpy as np
import pandas as pd
import re

# directories
clean = 'Clean_Data/'
raw = 'Foundation/Raw Data/'
root = '/root/'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Gdrive path
# file_path = '/content/drive/MyDrive/Project_508/Foundation/Raw Data/Other/Vehicle Population.xlsx'
file_path = root + 'Foundation/Raw Data/Other/Vehicle Population.xlsx'
veh_df = pd.read_excel(file_path,
                       sheet_name='ZIP')

# print shape
print(f'Shape {veh_df.shape}\n', '--'*20)

veh_df.head(3)

Shape (175291, 4)
 ----------------------------------------


,Data Year,Fuel Type,ZIP,Number of Vehicles
0,2010,Bio Diesel,90010,3
1,2010,Bio Diesel,90019,1
2,2010,Bio Diesel,90047,1


In [6]:
veh_df.rename(columns={'Data Year':'Year',
                                        'Fuel Type':'Fuel',
                                        'Number of Vehicles':'#_vehicles'},
                               inplace=True)

In [7]:
veh_df[veh_df['#_vehicles'] == 0]

,Year,Fuel,ZIP,#_vehicles


In [14]:
years = set(veh_df['Year'])

dfs = []
for year in years:
    print(f'Processesing {year}', '\n')

    year_df = veh_df.loc[veh_df['Year'] == year]

    zip_codes = set(year_df['ZIP'])

    for zip_code in zip_codes:

        mutate_df = year_df.loc[year_df['ZIP'] == zip_code, ['Fuel', '#_vehicles']]

        indexes = list(mutate_df.T.iloc[0,:])

        mutate_df.index = pd.Index(indexes,name='Fuel')

        mutate_df = mutate_df.T.reset_index(drop=True)

        mutate_df['Year'] = year
        mutate_df['ZIP'] = zip_code

        dfs.append(pd.DataFrame(mutate_df.iloc[1,:]).T)

final_df = pd.concat(dfs)
final_df.head(3)

Processesing 2016 

Processesing 2017 

Processesing 2018 

Processesing 2019 

Processesing 2020 

Processesing 2010 

Processesing 2011 

Processesing 2012 

Processesing 2013 

Processesing 2014 

Processesing 2015 



Index is labeld as "FUEL".  This will be omitted when DataFrame is written to CSV/Parquet.

In [34]:
final_df = final_df.fillna(0)

In [ ]:
clean_data_dir = root + clean

final_df.to_parquet(clean_data_dir + 'vehicle_registration.parquet.gzip',
                                   compression='gzip',
                                   index=False)

In [ ]:
print('Vehicle Data is READY TO ROCK!!!!')

In [ ]:
import boto3
import sagemaker

# Sagemaker session
sess = sagemaker.Session()

# bucket = "my-bucket"
bucket = sess.default_bucket()

file_name = 'vehicle_registration.parquet.gzip'
file_path = 'Clean_Data/' + file_name

# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(clean_data_dir + file_name, bucket, file_path)